https://pypi.python.org/pypi/windrose/

https://github.com/python-windrose/windrose

http://nbviewer.jupyter.org/github/python-windrose/windrose/blob/master/windrose_sample_random.ipynb

In [ ]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')


import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import createFolder as cF
import datetime

import get_Haukeli_obs_data as obsDat
import calc_date as cd
import netCDF4
import fill_values as fv
import save_fig as sF

from windrose_edit import WindroseAxes

In [ ]:
savefig = 0

stn = 'Haukeliseter' #
year= '2016'
month = '12'

hh = '00'

#cF.createFolder('../../Data/Soundings/%s_txt/' %(stn))      # for text files
figdir = '../../Figures/Windrose/%s/MEPS_weathermast' %(stn)
form = 'png'
cF.createFolder(figdir)      # for figure files
t = ['17', '18','19', '20', '21', '22', '23', '24', '25', '26', '27']
#t = ['20']

In [ ]:
level = '10m'
### MEPS
ncdir_10m = '../../Data/MEPS/%s/%s_%s' %(stn,level,hh)

### Haukeli obs
txtdir = '../../Data/eklima/'
txt_filename = 'Observations-hour_minutes.txt'

In [ ]:
# read in the Haukeliseter observation file, sorted by daily values
Haukeli = pd.read_csv('%s/%s' %(txtdir, txt_filename),\
                    sep = ';',header=19, skipfooter = 11, engine= 'python')

DateHour = Haukeli['Date-Hour(UTC)']

DD   = Haukeli['DD'].astype(float)             # Wind direction (FF); [degrees]
FF   = Haukeli['FF'].astype(float)             # Wind speed (10 meters above ground);m/s

In [ ]:
idxFF = np.where(FF[:] == -9999)
idxDD = np.where(DD[:] == -9999)

FF[idxFF[0][:]] = np.nan
DD[idxDD[0][:]] = np.nan


In [ ]:
ws_WM, wd_WM, dt_wind, hour_wind, day_wind, month_wind, year_wind = obsDat.valid_values_wind(FF, DD, DateHour)

In [ ]:
champ = 255.
date_blue = np.array([1,74,159])/255.
no1 = np.array([79,94,26])/champ
no2 = np.array([131,156,45])/champ
no3 = np.array([71,153,112])/champ
no4 = np.array([77,111,157])/champ
no5 = np.array([157,58,55])/champ
no6 = np.array([211,120,50])/champ
no7 = np.array([218,181,70])/champ

memb_col = np.array([99,99,99])/champ       # ensemble member color
dofe = np.array([64,180,233])/champ         # color for double fence measurement

In [ ]:
fn_10m = dict()

time_10m   = dict()
x_wind_10m = dict()
y_wind_10m = dict()

wd_MEPS    = dict()
ws_MEPS    = dict()

In [ ]:
for day in t:
    for ens_memb in range(0,10):
# read in the netcdf 10m file
        fn_10m[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_10m,year,month,day,hh,ens_memb))
        time_10m[ens_memb] = fn_10m[ens_memb].variables['time']
        time_10m[ens_memb] = fv.fill_nan(time_10m[ens_memb][:])
        
        ini_day = (datetime.datetime.utcfromtimestamp(time_10m[0][0]).day)   # day of initialisation
        hour = (datetime.datetime.utcfromtimestamp(time_10m[0][0]).hour)       # first hour of initialisation
        calday, calmon = cd.get_dayname(year, month, ini_day)

# Read in the variable name which should be plotted  
        x_wind_10m[ens_memb] = fn_10m[ens_memb].variables['x_wind_10m'][:]
        y_wind_10m[ens_memb] = fn_10m[ens_memb].variables['y_wind_10m'][:]
        
        x_wind_10m[ens_memb] = fv.fill_nan(x_wind_10m[ens_memb])
        y_wind_10m[ens_memb] = fv.fill_nan(y_wind_10m[ens_memb])
    
# Create wind speed and direction variables
        wd_MEPS[ens_memb] = np.arctan2(y_wind_10m[ens_memb][:,0], x_wind_10m[ens_memb][:,0])
        ws_MEPS[ens_memb] = np.sqrt((x_wind_10m[ens_memb][:,0])**2 + (y_wind_10m[ens_memb][:,0])**2) 

        wd_MEPS[ens_memb] = 270-np.rad2deg(wd_MEPS[ens_memb])  # convert from math to meteo direction and to degrees

        ws_MEPS[ens_memb] = ws_MEPS[ens_memb][~np.isnan(wd_MEPS[ens_memb])]
        wd_MEPS[ens_memb] = wd_MEPS[ens_memb][~np.isnan(wd_MEPS[ens_memb])]

        
#### plot figure
    figname = '%s%s%s.%s' %(year,month,day,form)
    
    levels = np.arange(0,28,4)
    cmap = mpl.colors.ListedColormap([no1, no2, no3, no4, no5, no6, no7])
    norm = mpl.colors.BoundaryNorm(boundaries = levels, ncolors=cmap.N)
    plt.hist([0,1])
    plt.close()
    
    fig = plt.figure(figsize=(20,10))
    ax0 = fig.add_subplot(121, projection='windrose')
    ax0.bar(wd_MEPS[0][:24], ws_MEPS[0][:24], normed=True, opening=1.0, edgecolor='white',nsector=45, 
                 cmap=cmap,bins=np.arange(0, 28, 4))
    ax0._info['bins']
    ax0.set_legend()
    ax0.legend(title ='Wind$_{10m}$ [m s$^{-1}$]',fontsize=14)
    ax0.get_legend().get_title().set_fontsize('18')
    ax0.tick_params(labelsize = 20)
    ax0.set_title('MEPS', fontsize=20,y=1.08, color=memb_col, fontweight='bold')
    
    ax1 = fig.add_subplot(122, projection='windrose')
    bar = ax1.bar(wd_WM[int(day)-1][:], ws_WM[int(day)-1][:], normed=True, opening=1.0, edgecolor='white',nsector=45, 
                 cmap=cmap,bins=np.arange(0, 28, 4))
    ax1._info['bins']
    ax1.set_title('Weather Mast', fontsize=20,y=1.08, color=dofe, fontweight='bold')
    fig.suptitle('%s, %s %s %s %s UTC' %(calday,ini_day,calmon,year,hh), color = date_blue,fontsize = 24)
    ax1.tick_params(labelsize = 20)
    
    if savefig == 1:
        sF.save_figure_portrait(figdir,figname,form)
        print('saved: %s/%s' %(figdir,figname) )
    else:
        plt.show()
    plt.close()
    
    for ens_memb in range(0,10):
        fn_10m[ens_memb].close()